In [1]:
import os
import numpy as np
os.chdir("/yourpath/CelebA")


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,0" #list the gpu cores in the order you want to use e.g "0,1,2,3"

In [3]:
f=open('list_attr_celeba.txt').read()

In [4]:
Ft=f.split('\n')[1].split()

In [6]:
X=np.array([r.split() for r in f.split('\n')[2:-1]])
ids=X[:,0]

In [7]:
X.shape

(202599, 41)

## to load for CelebA-Age use the block below 

In [8]:
g=X[:,Ft.index('Young')+1]
y=np.array([X[:,i+1] for i in range(len(Ft)) if i!=Ft.index('Young')])
y=[[0 if t[i]=='-1' else 1 for i in range(len(t))] for t in y]
g=np.array([0 if g[i]=='-1' else 1 for i in range(len(g))])
no_tasks=[4,10,17,13,14,17,26,29,35]
y=np.array([y[t] for t in range(len(y)) if t not in no_tasks])
Ft.remove('Young')

## to load for CelebA-Gender use the block below instead of the block above

In [ ]:
g=X[:,Ft.index('Male')+1]
y=np.array([X[:,i+1] for i in range(len(Ft)) if i!=Ft.index('Male')])
y=[[0 if t[i]=='-1' else 1 for i in range(len(t))] for t in y]
g=np.array([0 if g[i]=='-1' else 1 for i in range(len(g))])
tasks=[2,3,5,6,7,8,11,12,19,20,22,24,26,30,31,32,38]
y=np.array([y[t] for t in tasks])
Ft.remove('Male')

## From here onwards all the blocks are common for both CelebA-Gender and CelebA-Age

In [15]:
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.models as pretrain
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [16]:
dataroot = "/yourpath/CelebA/celeba/"
image_size = 64
batch_size = 8192
workers = 2

dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

In [20]:
model_pre=pretrain.resnet18(pretrained=True)

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
cpu=torch.device('cpu')
dv=torch.device("cuda" if (torch.cuda.is_available() ) else "cpu")#
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
indices=np.arange(len(ids))
in_tr,in_val,in_ts=indices[:162770],indices[162770:182637],indices[182637:]
in_tr1=in_tr[:5*batch_size]; in_tr2=in_tr[5*batch_size:10*batch_size]; in_tr3=in_tr[10*batch_size:15*batch_size]
in_tr4=in_tr[15*batch_size:]
in_val1,in_val2=in_val[:10000],in_val[10000:]

dataset_tr1 = torch.utils.data.Subset(dataset, in_tr1)
dataset_tr2 = torch.utils.data.Subset(dataset, in_tr2)
dataset_tr3 = torch.utils.data.Subset(dataset, in_tr3)
dataset_tr4 = torch.utils.data.Subset(dataset, in_tr4)
dataset_val1 = torch.utils.data.Subset(dataset, in_val1)
dataset_val2 = torch.utils.data.Subset(dataset, in_val2)
dataset_ts = torch.utils.data.Subset(dataset, in_ts)

dataloader1 = torch.utils.data.DataLoader(dataset_tr1, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)
dataloader2 = torch.utils.data.DataLoader(dataset_tr2, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)
dataloader3 = torch.utils.data.DataLoader(dataset_tr3, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)
dataloader4 = torch.utils.data.DataLoader(dataset_tr4, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)

dataloader_val1 = torch.utils.data.DataLoader(dataset_val1, batch_size=10000,
                                         shuffle=False, num_workers=workers)
dataloader_val2 = torch.utils.data.DataLoader(dataset_val2, batch_size=10000,
                                         shuffle=False, num_workers=workers)

dataloader_test = torch.utils.data.DataLoader(dataset_ts, batch_size=len(in_ts),
                                         shuffle=False, num_workers=workers)


y_test=[y[i][in_ts] for i in range(len(y))]

y_v1,y_v2=[y[i][in_val1] for i in range(len(y))],[y[i][in_val2] for i in range(len(y))]

y_tr1,y_tr2=[y[i][in_tr1] for i in range(len(y))],[y[i][in_tr2] for i in range(len(y))]
y_tr3,y_tr4=[y[i][in_tr3] for i in range(len(y))],[y[i][in_tr4] for i in range(len(y))]



g_tr1,g_tr2,g_tr3,g_tr4=g[in_tr1],g[in_tr2],g[in_tr3],g[in_tr4]
g_val1,g_val2=g[in_val1],g[in_val2]
g_test=g[in_ts]
N_tasks=len(y)#4#2#3

In [31]:
N_tasks

31

In [33]:
d_v=[dataloader_val1,dataloader_val2]
yv=[y_v1,y_v2]
gv=[g_val1,g_val2]

In [28]:
for par in model_pre.parameters():
    par.requires_grad=False
in_fc=model_pre.fc.in_features
model_pre.fc=nn.Linear(in_fc,1024)

In [30]:
class MTL(nn.Module):

    def __init__(self,pretrain,tasks=2):
        super(MTL, self).__init__()
        self.tasks=tasks
        self.fc1 = pretrain
        self.bn1= nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 128)
        
        self.bn3 = nn.BatchNorm1d(1024)
        self.fc4 = nn.Linear(1024,1024)
        self.tasks_out=nn.ModuleDict({str(t):nn.Linear(128,2) for t in range(self.tasks)})

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        
        x = F.relu(self.bn3(self.fc4(x)))
        
        x = F.relu(self.bn2(self.fc2(x)))        
        x = F.relu(self.fc3(x))

        t=[self.tasks_out[str(i)](x) for i in range(self.tasks)]
        
        return t

In [26]:
def fair_loss(output,target,x_control):
    prot_att=x_control
    index_prot=torch.squeeze(torch.nonzero(prot_att[:] != 1.))
    target_prot=torch.index_select(target, 0, index=index_prot)
    index_prot_pos=torch.squeeze(torch.nonzero(target_prot[:] == 1. ))
    index_prot_neg=torch.squeeze(torch.nonzero(target_prot[:] == 0. ))

    index_non_prot=torch.squeeze(torch.nonzero(prot_att[:] == 1.))
    target_non_prot=torch.index_select(target, 0, index=index_non_prot)
    index_non_prot_pos=torch.squeeze(torch.nonzero(target_non_prot[:] == 1. ))
    index_non_prot_neg=torch.squeeze(torch.nonzero(target_non_prot[:] == 0. ))

    l_prot_pos=F.cross_entropy(torch.index_select(output, 0, index=index_prot_pos),torch.index_select(target, 0, index=index_prot_pos))    
    l_non_prot_pos=F.cross_entropy(torch.index_select(output, 0, index=index_non_prot_pos),torch.index_select(target, 0, index=index_non_prot_pos))    
    l_non_prot_neg=F.cross_entropy(torch.index_select(output, 0, index=index_non_prot_neg),torch.index_select(target, 0, index=index_non_prot_neg))
    l_prot_neg=F.cross_entropy(torch.index_select(output, 0, index=index_prot_neg),torch.index_select(target, 0, index=index_prot_neg))    

    for l in [l_prot_pos,l_non_prot_pos,l_prot_neg,l_non_prot_neg]:
        if torch.isinf(l)==True:
            l=torch.zeros_like(l,requires_grad=True)
    dl_pos=torch.max(l_prot_pos,l_non_prot_pos)
    dl_neg=torch.max(l_prot_neg,l_non_prot_neg)
    L=dl_pos+dl_neg
    
    return L

In [31]:
import torchmetrics
acc = torchmetrics.Accuracy()
def DM_rate(output,target,x_control):
    prot_att=x_control
    index_prot=torch.squeeze(torch.nonzero(prot_att[:] != 1.))
    target_prot=torch.index_select(target, 0, index=index_prot)
    index_prot_pos=torch.squeeze(torch.nonzero(target_prot[:] == 1. ))
    index_prot_neg=torch.squeeze(torch.nonzero(target_prot[:] == 0. ))

    index_non_prot=torch.squeeze(torch.nonzero(prot_att[:] == 1.))
    target_non_prot=torch.index_select(target, 0, index=index_non_prot)
    index_non_prot_pos=torch.squeeze(torch.nonzero(target_non_prot[:] == 1. ))
    index_non_prot_neg=torch.squeeze(torch.nonzero(target_non_prot[:] == 0. ))

    if index_prot_pos.shape==torch.Size([]) or index_prot_pos.shape==torch.Size([0])\
        or index_non_prot_pos.shape==torch.Size([]) or index_non_prot_pos.shape==torch.Size([0]):
            l_prot_pos=torch.tensor(0.0001)
            l_non_prot_pos=torch.tensor(0.0001)
    else:        
            l_prot_pos=acc(torch.index_select(output, 0, index=index_prot_pos),torch.index_select(target, 0, index=index_prot_pos))    
            l_non_prot_pos=acc(torch.index_select(output, 0, index=index_non_prot_pos),torch.index_select(target, 0, index=index_non_prot_pos))    
    
    if index_prot_neg.shape==torch.Size([]) or index_prot_neg.shape==torch.Size([0])\
        or index_non_prot_neg.shape==torch.Size([]) or index_non_prot_neg.shape==torch.Size([0]):
            l_prot_neg=torch.tensor(0.0001)
            l_non_prot_neg=torch.tensor(0.0001)
    else:        
            l_prot_neg=acc(torch.index_select(output, 0, index=index_prot_neg),torch.index_select(target, 0, index=index_prot_neg))    
            l_non_prot_neg=acc(torch.index_select(output, 0, index=index_non_prot_neg),torch.index_select(target, 0, index=index_non_prot_neg))  
            
    dl_pos=torch.abs(l_prot_pos-l_non_prot_pos)
    dl_neg=torch.abs(l_prot_neg-l_non_prot_neg)
    DM=dl_pos+dl_neg
    
    return DM

In [41]:
class DeepQNetwork(nn.Module):
    def __init__(self, input_dims, fc1_dims, fc2_dims,fc3_dims=256,
                 n_actions=2,n_tasks=2):
        super(DeepQNetwork, self).__init__()
        self.tasks=n_tasks
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.bn1= nn.LayerNorm(fc1_dims)
        self.fc2_dims = fc2_dims
        self.bn2= nn.LayerNorm(fc2_dims)
        self.fc3_dims = fc3_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.fc3_dims)
        self.fc_out=nn.ModuleDict({str(t):nn.Linear(self.fc3_dims, self.n_actions) for t in range(self.tasks)})

    def forward(self, state):
        x = F.relu(self.bn1(self.fc1(state)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.relu(self.fc3(x))

        Q_sa=[self.fc_out[str(i)](x) for i in range(self.tasks)]

        return Q_sa#Q1_sa,Q2_sa#,Q3_sa

In [42]:

gamma=0.9
omega_T=torch.tensor([1/N_tasks for t in range(N_tasks)]).to(dv)
init_loss_T=[None for t in range(N_tasks)]
criteria_t=nn.MSELoss()
def Teacher(state,state_a,state_b,R_a,R_b,DqN,opti_T):
    grads,G_n,loss_ratio=[],[],[]
    global init_loss_T
    global omega_T
    opti_T.zero_grad()
    for t in range(N_tasks):        
        Q_S=DqN(state[t])[t]
        target=[]
        with torch.no_grad():
            Q_sa=R_a[t]+gamma*torch.max(DqN(state_a[t])[t],1)[0]
            Q_sb=R_b[t]+gamma*torch.max(DqN(state_b[t])[t],1)[0]
        target=torch.tensor([[Q_sa[i],Q_sb[i]] for i in range(len(Q_sa))]).to(dv)
        teach_loss=criteria_t(Q_S,target)
        teach_loss.backward()
        if init_loss_T[t]== None:
            init_loss_T[t]=teach_loss.item()
        loss_ratio.append(teach_loss.item()/init_loss_T[t])
        grads_sh={}
        for n,p in DqN.named_parameters():
            if p.data.shape[0]!=2 and p.grad!=None:
                grads_sh[n] = p.grad 
                
        grads.append(grads_sh)
        G_n.append(torch.linalg.norm(torch.stack([torch.linalg.norm(grads_sh[g]) for g in grads_sh])))
    G_n = torch.stack(G_n)
    E_t = sum(loss_ratio)/len(loss_ratio)
    r_t=[loss/E_t for loss in loss_ratio] #relative inverse trainin g rate of teacher
    omega_T,DqN = Update_model(DqN,grads,omega_T,G_n,r_t,opti_T)
    torch.save(omega_T,'/yourpath/CelebA/model/age/omega_T.pt') # 
                   
    return DqN

In [43]:
lr=0.001
def Update_model(model,grads_sh,omega,G_n,r_t,opti):
    loss_gn=[(G_n[t]-torch.mean(G_n)*r_t[t]) for t in range(len(G_n))]
    for i in range(len(G_n)):
        d_l=0
        if loss_gn[i]>0:
            d_l+=(len(G_n)-1)/len(G_n)*G_n[i]
        elif loss_gn[i]<0:
            d_l-=(len(G_n)-1)/len(G_n)*G_n[i]
        for j in range(len(G_n)):
            if j!=i:
                if loss_gn[j]>0:
                    d_l-=(G_n[i]/len(G_n))
                elif loss_gn[j]<0:
                    d_l+=(G_n[i]/len(G_n))
        
        omega[i]-=lr*d_l
        omega[i]=1
    for t in range(len(G_n)):
        for n,p in model.named_parameters():
            if p.data.shape[0]!=2 and p.grad!=None:
                if t==0:
                    p.grad=omega[t]*grads_sh[t][n]
                else:
                    p.grad+=omega[t]*grads_sh[t][n]
                    
    opti.step() 
    for i in range(len(omega)):
        if omega[i]<0:
            omega[i]=-omega[i]
    return omega,model

In [44]:
net=MTL(model_pre.to(dv),tasks=N_tasks)
net=nn.DataParallel(net)
net.to(dv)
opti_S=optim.AdamW(params=net.parameters())

flat_w=torch.flatten(net.module.tasks_out['0'].weight)
dqn=DeepQNetwork(len(flat_w),512,512,n_actions=2,n_tasks=N_tasks).to(dv)
opti_T = optim.AdamW(dqn.parameters())

In [46]:
def Student(stud,path,action,X,y,t=0,xc=None):
    stud.load_state_dict(torch.load(path))
    opti = optim.Adam(stud.parameters())
    opti.zero_grad()
    out=stud(X)[t]
    if xc!=None:
        loss=action(out,y,xc)        
    else:
        loss=action(out,y)
    #print(loss)
    loss.backward()
    opti.step()
    return stud

In [47]:
omega_S=torch.tensor([1/N_tasks for t in range(N_tasks)]).to(dv)
init_loss_S=[None for t in range(N_tasks)]
m_acc,flag=0.0,1
criteria = nn.CrossEntropyLoss()
best_S=[[0,0] for t in range(N_tasks)]
optim_path,optim_path_disc,optim_clas=[],[],[]
path='/yourpath/CelebA/model/age/Model_l2tfmt.pt' #path to save your model
path_t='/yourpath/CelebA/model/age/dqn_n.pt' ##path to save your teacher model
torch.save(net.state_dict(),path)
action_choices=[]
stud=nn.DataParallel(MTL(model_pre,tasks=N_tasks)).to(dv)
dt=[dataloader1,dataloader2,dataloader3,dataloader4]
yt=[y_tr1,y_tr2,y_tr3,y_tr4]
gt=[g_tr1,g_tr2,g_tr3,g_tr4]
for epoch in range(20):  # loop over each NN multiple times
    loss_pointer=[]
    state_net,state_a,state_f=[[] for t in range(N_tasks)],[[] for t in range(N_tasks)],[[] for t in range(N_tasks)]
    R_A,R_F=[[] for t in range(N_tasks)],[[] for t in range(N_tasks)]
    ch=np.random.choice([0,1,2,3])
    dataloader,y_train,g_train=dt[ch],yt[ch],gt[ch]
    y_train=[torch.tensor(y_train[i]) for i in range(N_tasks)]
    xg=torch.tensor(g_train)
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs =data[0]
        if ((i+1)*batch_size)<len(in_tr):            
            labels=[y_train[t][i*batch_size:(i+1)*batch_size] for t in range(N_tasks)]
            xc=xg[i*batch_size:(i+1)*batch_size]
        else:
            labels=[y_train[t][i*batch_size:] for t in range(N_tasks)]
            xc=xg[i*batch_size:]
        
        # zero the parameter gradients
        net.load_state_dict(torch.load(path))
        net.to(dv)
        fg=[0 for i in range(N_tasks)]
        with torch.no_grad():
            for t in range(N_tasks):
                flat=torch.flatten(net.module.tasks_out[str(t)].weight)
                fg[t]=torch.argmax(dqn(flat)[t])
                       
        opti_S.zero_grad()
        outputs = net(inputs.to(dv).float())
        lp,grads,G_n,loss_ratio=[0 for t in range(N_tasks)],[],[],[]
        for t in range(N_tasks):
            
            #MTL starts
            loss_a=criteria(outputs[t], labels[t].to(dv))
            loss_f=fair_loss(outputs[t], labels[t].to(dv),xc.to(dv))
            if fg[t]==0:
                loss_t=loss_a
                lp[t]=0
            else:
                loss_t=loss_f
                lp[t]=1
            loss_t.backward(retain_graph=True)
            
            if init_loss_S[t]== None:
                init_loss_S[t]=loss_t.item()
            loss_ratio.append(loss_t.item()/init_loss_S[t])
            grads_sh={}
            for n,p in net.named_parameters():
                if p.data.shape[0]!=2 and p.grad!=None:
                    grads_sh[n] = p.grad
                    p.grad=None
            grads.append(grads_sh)
            G_n.append(torch.linalg.norm(torch.stack([torch.linalg.norm(grads_sh[g]) for g in grads_sh])))
        loss_pointer.append(lp)
        G_n = torch.stack(G_n)
        E_t = sum(loss_ratio)/len(loss_ratio)
        r_t=[loss/E_t for loss in loss_ratio] #relative inverse trainin g rate of student
        omega_S,net = Update_model(net,grads,omega_S,G_n,r_t,opti_S) 
        torch.save(net.state_dict(),path)
        torch.save(omega_S,'/yourpath/CelebA/model/age/omega_S.pt')
        for t in range(N_tasks):
            state_net[t].append(torch.unsqueeze(torch.flatten(net.module.tasks_out[str(t)].weight),dim=0))

            accuracy=acc(outputs[t].to(cpu),labels[t].to(cpu))
            fairness=DM_rate(outputs[t].to(cpu), labels[t].to(cpu),xc.to(cpu))

            learn_a=Student(stud,path,criteria,inputs.to(dv).float(),labels[t].to(dv),t=t)

            learn_f=Student(stud,path,fair_loss,inputs.to(dv).float(),labels[t].to(dv),t=t,xc=xc.to(dv))


            state_a[t].append(torch.unsqueeze(torch.flatten(learn_a.module.tasks_out[str(t)].weight),dim=0))            
            state_f[t].append(torch.unsqueeze(torch.flatten(learn_f.module.tasks_out[str(t)].weight),dim=0))
        
            with torch.no_grad():
                out_A=learn_a(inputs.to(dv).float())[t]
                out_F=learn_f(inputs.to(dv).float())[t]
                acc_a=acc(out_A.to(cpu),labels[t].to(cpu))
                DM_a=DM_rate(out_A.to(cpu),labels[t].to(cpu),xc.to(cpu))
                acc_f=acc(out_F.to(cpu),labels[t].to(cpu))
                DM_f=DM_rate(out_F.to(cpu),labels[t].to(cpu),xc.to(cpu))
        
            if best_S[t][0]==0:
                best_S[t][0]=accuracy.item()
                best_S[t][1]=fairness
            # the reward functions
            R_A[t].append(torch.min((acc_a-best_S[t][0])/best_S[t][0],(1-DM_a-best_S[t][1])/best_S[t][1]))
            R_F[t].append(torch.min((acc_f-best_S[t][0])/best_S[t][0],(1-DM_f-best_S[t][1])/best_S[t][1]))
        del inputs; del labels; del xc 
    for t in range(N_tasks):
        state_net[t]=torch.cat(state_net[t],0).to(dv)
        state_a[t]=torch.cat(state_a[t],0).to(dv)
        state_f[t]=torch.cat(state_f[t],0).to(dv)
        R_A[t]=torch.tensor(R_A[t]).to(dv)
        R_F[t]=torch.tensor(R_F[t]).to(dv)
    dqn=Teacher(state_net,state_a,state_f,R_A,R_F,dqn,opti_T)
    torch.save(dqn.state_dict(),path_t)
    del state_net; del state_a; del state_f; del R_A; del R_F
    torch.cuda.empty_cache()
    with torch.no_grad():
        ch=np.random.choice([0,1])
        val_batch,y_v,g_val=next(iter(d_v[ch]))[0],yv[ch],gv[ch]
        pred0=net(val_batch.to(dv))#=Val_mod
        print('Epoch: ',epoch, 'Loss pointers',loss_pointer)
        action_choices.append(loss_pointer)
        np.save(file="/yourpath/CelebA/loss_pointers_age_n.npy",arr=np.array(action_choices)) 
        for t in range(N_tasks): 
            accuracy=acc(pred0[t].to(cpu),torch.tensor(y_v[t]))
            EO=DM_rate(pred0[t].to(cpu),torch.tensor(y_v[t]),torch.tensor(g_val))
            
            print('Task',t,'  Acc:',accuracy, 'EO:',EO)
            if accuracy>best_S[t][0]:
                best_S[t][0]=accuracy
            if 1-EO>best_S[t][1]:
                best_S[t][1]=1-EO
        

Epoch:  14 Loss pointers [[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0], [1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0], [1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0]]
Task 0   Acc: tensor(0.8883) EO: tensor(0.0325)
Task 1   Acc: tensor(0.7299) EO: tensor(0.0248)
Task 2   Acc: tensor(0.5923) EO: tensor(0.0250)
Task 3   Acc: tensor(0.7929) EO: tensor(0.0090)
Task 4   Acc: tensor(0.8526) EO: tensor(0.0130)
Task 5   Acc: tensor(0.7529) EO: tensor(0.0101)
Task 6   Acc: tensor(0.7696) EO: tensor(0.0146)
Task 7   Acc: tensor(0.7696) EO: tensor(0.0058)
Task 8   Acc: tensor(0.8484) EO: tensor(0.0037)
Task 9   Acc: tensor(0.7894) EO: tensor(0.0220)
Task 10   Acc: tensor(0.8641) EO: tensor(0.0121)
Task 11   Acc: tensor(0.9333) EO: tensor(0.0165)
Task 12   Acc: tensor(0.9397) EO: tensor(0.0177)
Task 13   Acc: tensor(0.6138) EO: tensor(0.0181)
Task 14   Acc

Task 18   Acc: tensor(0.8794) EO: tensor(0.0244)
Task 19   Acc: tensor(0.8411) EO: tensor(0.0229)
Task 20   Acc: tensor(0.7217) EO: tensor(0.0107)
Task 21   Acc: tensor(0.7141) EO: tensor(0.0123)
Task 22   Acc: tensor(0.9185) EO: tensor(0.0110)
Task 23   Acc: tensor(0.9455) EO: tensor(0.0085)
Task 24   Acc: tensor(0.5711) EO: tensor(0.0210)
Task 25   Acc: tensor(0.7975) EO: tensor(0.0273)
Task 26   Acc: tensor(0.6747) EO: tensor(0.0068)
Task 27   Acc: tensor(0.8095) EO: tensor(0.0168)
Task 28   Acc: tensor(0.6602) EO: tensor(0.0057)
Task 29   Acc: tensor(0.8747) EO: tensor(0.0475)
Task 30   Acc: tensor(0.9285) EO: tensor(0.0270)


In [64]:
class STL(nn.Module):

    def __init__(self,pretrain):
        super(STL, self).__init__()
        self.fc1 = pretrain 
        self.bn1= nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 128)
        
        self.bn3 = nn.BatchNorm1d(1024)
        self.fc4 = nn.Linear(1024,1024)     
        self.task = nn.Linear(128,2)


    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        
        x = F.relu(self.bn3(self.fc4(x)))
        
        x = F.relu(self.bn2(self.fc2(x)))        
        x = F.relu(self.fc3(x))
        t = self.task(x)
       
        return t

In [32]:
spaths={'path'+str(t):'/yourpath/CelebA/model/Model_stl'+str(t)+'.pt' for t in range(N_tasks)}
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
test_batch=next(iter(dataloader_test))[0]

In [33]:
snet=nn.DataParallel(STL(model_pre.to(device)))
snet.to(device)
Bests=[]
for t in range(N_tasks):  
    snet.load_state_dict(torch.load(spaths['path'+str(t)]))
    pred0=snet(test_batch.to(device).float())
    accuracy=acc(pred0.to(cpu),torch.tensor(y_test[t]).to(cpu))
    DM=DM_rate(pred0.to(cpu),torch.tensor(y_test[t]).to(cpu),torch.tensor(g_test).to(cpu))
    Bests.append([accuracy,DM])
    print('Task',t,'  Acc:',accuracy, 'DMs:',DM)

Task 0   Acc: tensor(0.8859) DMs: tensor(0.0108)
Task 1   Acc: tensor(0.7613) DMs: tensor(0.0206)
Task 2   Acc: tensor(0.7063) DMs: tensor(0.0190)
Task 3   Acc: tensor(0.7929) DMs: tensor(0.0137)
Task 4   Acc: tensor(0.8771) DMs: tensor(0.0115)
Task 5   Acc: tensor(0.7701) DMs: tensor(0.0136)
Task 6   Acc: tensor(0.7801) DMs: tensor(0.0103)
Task 7   Acc: tensor(0.8116) DMs: tensor(0.0121)
Task 8   Acc: tensor(0.9135) DMs: tensor(0.0111)
Task 9   Acc: tensor(0.8126) DMs: tensor(0.0416)
Task 10   Acc: tensor(0.8565) DMs: tensor(0.0164)
Task 11   Acc: tensor(0.9409) DMs: tensor(0.0115)
Task 12   Acc: tensor(0.9385) DMs: tensor(0.0385)
Task 13   Acc: tensor(0.7935) DMs: tensor(0.0015)
Task 14   Acc: tensor(0.6843) DMs: tensor(0.0263)
Task 15   Acc: tensor(0.8323) DMs: tensor(0.0165)
Task 16   Acc: tensor(0.6191) DMs: tensor(0.0162)
Task 17   Acc: tensor(0.9594) DMs: tensor(0.0106)
Task 18   Acc: tensor(0.8792) DMs: tensor(0.0065)
Task 19   Acc: tensor(0.8409) DMs: tensor(0.0046)
Task 20   

In [34]:
del snet
torch.cuda.empty_cache()

In [35]:
net=MTL(model_pre.to(dv),tasks=N_tasks)
net=nn.DataParallel(net)
net.to(dv)

DataParallel(
  (module): MTL(
    (fc1): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(6

In [37]:
l2t=[]
net.load_state_dict(torch.load('/yourpath/CelebA/model/age/Model_multi.pt'))
pred0=net(test_batch.to(device).float())
ra,rf=0,0
for t in range(N_tasks):         
    accuracy=acc(pred0[t].to(cpu),torch.tensor(y_test[t]).to(cpu))
    DM=DM_rate(pred0[t].to(cpu),torch.tensor(y_test[t]).to(cpu),torch.tensor(g_test).to(cpu)) 
    l2t.append([accuracy,DM])
    ra+=accuracy/Bests[t][0]
    rf+=DM/Bests[t][1]
    print('Task',t,'  Acc:',accuracy, 'EOs:',DM)
    print('Relative:',ra/(t+1), rf/(t+1))

Task 0   Acc: tensor(0.8876) EOs: tensor(0.0050)
Relative: tensor(1.0019) tensor(0.4657)
Task 1   Acc: tensor(0.7363) EOs: tensor(0.0074)
Relative: tensor(0.9845) tensor(0.4118)
Task 2   Acc: tensor(0.5872) EOs: tensor(0.0070)
Relative: tensor(0.9334) tensor(0.3974)
Task 3   Acc: tensor(0.7950) EOs: tensor(0.0225)
Relative: tensor(0.9507) tensor(0.7088)
Task 4   Acc: tensor(0.8466) EOs: tensor(0.0270)
Relative: tensor(0.9536) tensor(1.0353)
Task 5   Acc: tensor(0.7637) EOs: tensor(0.0031)
Relative: tensor(0.9600) tensor(0.9007)
Task 6   Acc: tensor(0.7638) EOs: tensor(0.0139)
Relative: tensor(0.9627) tensor(0.9652)
Task 7   Acc: tensor(0.7615) EOs: tensor(0.0207)
Relative: tensor(0.9596) tensor(1.0589)
Task 8   Acc: tensor(0.8550) EOs: tensor(0.0276)
Relative: tensor(0.9570) tensor(1.2180)
Task 9   Acc: tensor(0.7924) EOs: tensor(0.0348)
Relative: tensor(0.9588) tensor(1.1799)
Task 10   Acc: tensor(0.8555) EOs: tensor(0.0147)
Relative: tensor(0.9625) tensor(1.1542)
Task 11   Acc: tenso